In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

In [2]:
lbt_itp_id = 170

In [3]:
%%sql -o lbt_positions

# get all vehicle positions on July 12th, for the feed with itp id 170, and url number 0
SELECT *
FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
WHERE _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-10-21_170_0.parquet"
    # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-10-22_170_0.parquet"
    # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-10-23_170_0.parquet"
    # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-10-24_170_0.parquet"
ORDER BY header_timestamp

,calitp_itp_id,calitp_url_number,entity_id,vehicle_id,vehicle_trip_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,170,0,2410,None,9288757,1634793021,33.914220,-118.099250,1634793048
1,170,0,2415,None,9288449,1634793013,33.802090,-118.125175,1634793048
2,170,0,2421,None,9288542,1634793005,33.831257,-118.125270,1634793048
3,170,0,2423,None,9288541,1634793045,33.775430,-118.136720,1634793048
4,170,0,2427,None,9288758,1634792987,33.788647,-118.107635,1634793048
...,...,...,...,...,...,...,...,...,...
263121,170,0,1834,None,9289014,1634860729,33.831330,-118.075590,1634860782
263122,170,0,1836,None,9288352,1634860743,33.875736,-118.214750,1634860782
263123,170,0,1837,None,9288172,1634860745,33.846820,-118.195170,1634860782
263124,170,0,1838,None,9289461,1634860768,33.811203,-118.185104,1634860782


In [11]:
min_date, max_date = ('2021-10-21', '2021-10-21')
pch_routes = ('171', '172', '173', '174', '175')

lbt_trips = (tbl.views.gtfs_schedule_fact_daily_trips()
    # >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
    >> select(_.calitp_itp_id, _.calitp_url_number,
              _.date == _.service_date,
              _.trip_key, _.trip_id, _.route_id,
              _.is_in_service, _.calitp_extracted_at, _.calitp_deleted_at)
    # >> filter(_.route_id.isin(pch_routes))
    >> filter(_.calitp_itp_id == lbt_itp_id)
    >> filter(_.date == min_date)
    >> filter(_.is_in_service == True)
    >> collect()
    )

In [16]:
lbt_trips >> filter(_.route_id.isin(pch_routes)) >> inner_join(_, (lbt_positions >> rename(trip_id = 'vehicle_trip_id')), on='trip_id')

,calitp_itp_id_x,calitp_url_number_x,date,trip_key,trip_id,route_id,is_in_service,calitp_extracted_at,calitp_deleted_at,calitp_itp_id_y,calitp_url_number_y,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,170,0,2021-10-21,5974160129393889478,9288706,173,True,None,None,170,0,2521,None,1634823952,33.782803,-118.16878,1634823959
1,170,0,2021-10-21,5974160129393889478,9288706,173,True,None,None,170,0,2521,None,1634823952,33.782803,-118.16878,1634823989
2,170,0,2021-10-21,5974160129393889478,9288706,173,True,None,None,170,0,2521,None,1634823952,33.782803,-118.16878,1634823989
3,170,0,2021-10-21,5974160129393889478,9288706,173,True,None,None,170,0,2521,None,1634824012,33.782757,-118.16878,1634824019
4,170,0,2021-10-21,5974160129393889478,9288706,173,True,None,None,170,0,2521,None,1634824012,33.782757,-118.16878,1634824049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39679,170,0,2021-10-21,-2759441160519375861,9289779,172,True,None,None,170,0,2519,None,1634833281,33.865580,-118.09656,1634833322
39680,170,0,2021-10-21,-2759441160519375861,9289779,172,True,None,None,170,0,2519,None,1634833281,33.865580,-118.09656,1634833322
39681,170,0,2021-10-21,-2759441160519375861,9289779,172,True,None,None,170,0,2519,None,1634833343,33.865580,-118.09655,1634833352
39682,170,0,2021-10-21,-2759441160519375861,9289779,172,True,None,None,170,0,2519,None,1634833343,33.865580,-118.09655,1634833382


In [7]:
lbt_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   calitp_itp_id        1680 non-null   int64 
 1   calitp_url_number    1680 non-null   int64 
 2   date                 1680 non-null   object
 3   trip_key             1680 non-null   int64 
 4   trip_id              1680 non-null   object
 5   route_id             1680 non-null   object
 6   is_in_service        1680 non-null   bool  
 7   calitp_extracted_at  0 non-null      object
 8   calitp_deleted_at    0 non-null      object
dtypes: bool(1), int64(3), object(5)
memory usage: 106.8+ KB
